In [1]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime

In [2]:
def get_url(position, location):
        template = 'https://www.indeed.com/jobs?q={}&l={}'
        url = template.format(position, location)
        return url

In [3]:
url = get_url("data science", "la")

In [5]:
response = requests.get(url)
response

<Response [200]>

In [6]:
soup = BeautifulSoup(response.text, 'html.parser')
cards = soup.find_all('div', 'job_seen_beacon')
len(cards)

15

In [7]:
card = cards[1]

In [9]:
atag = card.h2.a
atag

<a aria-label="full details of Senior Data Analyst" class="jcs-JobTitle" data-hide-spinner="true" data-hiring-event="false" data-jk="dbbb13fd3da9d479" data-mobtk="1g34drf4dt55o800" href="/rc/clk?jk=dbbb13fd3da9d479&amp;fccid=bf75a171be6c8638&amp;vjs=3" id="job_dbbb13fd3da9d479" role="button" target="_blank"><span title="Senior Data Analyst">Senior Data Analyst</span></a>

In [10]:
job_title = atag.span.get("title")
job_title

'Senior Data Analyst'

In [11]:
job_url = 'https://www.indeed.com' + atag.get("href")
job_url

'https://www.indeed.com/rc/clk?jk=dbbb13fd3da9d479&fccid=bf75a171be6c8638&vjs=3'

In [12]:
company = card.find("span", {"class": "companyName"}).text
company

'Billion Dollar Boy'

In [13]:
job_location = card.find("div", {"class": "companyLocation"}).text
job_location

'New Orleans, LA 70115 (Touro area)'

In [14]:
job_summary = card.find("div", "job-snippet").text
job_summary

'\nAnalysing our own data to help drive strategic data driven decisions.\nExperience with Google Sheets, SQL, experience with dashboard tools a bonus (we utilize…\n'

In [15]:
post_date = card.find("span", "date").text
post_date

'Posted17 days ago'

In [16]:
today = datetime.today().strftime("%Y-%m-%d")
today

'2022-05-15'

## Gettting to the next page

In [18]:
page = soup.find("div", "pagination")
page.a.get("href")

'/jobs?q=data+science&l=la&start=10'

In [19]:
url = 'https://www.indeed.com' + page.a.get("href")
url

'https://www.indeed.com/jobs?q=data+science&l=la&start=10'

## Putting all together

In [23]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import csv

In [24]:
def get_url(position, location):
        template = 'https://www.indeed.com/jobs?q={}&l={}'
        url = template.format(position, location)
        return url

def get_record(card):
    
    job_title = atag.span.get("title")
    company = card.find("span", {"class": "companyName"}).text
    job_location = card.find("div", {"class": "companyLocation"}).text
    post_date = card.find("span", "date").text
    today = datetime.today().strftime("%Y-%m-%d")
    job_summary = card.find("div", "job-snippet").text
    job_url = 'https://www.indeed.com' + card.h2.a.get("href")
    
    """Below part is added to get the full job description"""
    response1 = requests.get(job_url)    
    soup1 = BeautifulSoup(response1.text, 'html.parser')

    jd = soup1.find('div', 'jobsearch-JobComponent-description icl-u-xs-mt--md')
    full_jd = jd.find("div", "jobsearch-jobDescriptionText").text
    
    
    record = (job_title, company, job_location, post_date, today, job_summary, job_url, full_jd)
    return record

def main(position, location):
    records = []
    url = get_url(position, location)
    
    for x in range(1):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'job_seen_beacon')
        
    
        for card in cards:
            record = get_record(card)
            records.append(record)
    
        try:
            url = 'https://www.indeed.com' + page.a.get("href")
        except attributeError:
            break
    
    with open('Scrapper_results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Job Title', 'Company', 'Location', 'Posting Date', 'Extract Date', 'Summary', 'Job Url', 'Job Description'])
        writer.writerows(records)

In [25]:
main("data engineer", "la")

In [26]:
import pandas as pd

df = pd.read_csv("Scrapper_results.csv")
df

,Job Title,Company,Location,Posting Date,Extract Date,Summary,Job Url,Job Description
0,Senior Data Analyst,Noodle,"Remote in New Orleans, LA 70118+1 location",Posted30+ days ago,2022-05-15,\nBuilding Tableau visualizations to make data...,https://www.indeed.com/rc/clk?jk=552e017fbbb4d...,\n\nData Analytics Engineer (Contract to Hire)...
1,Senior Data Analyst,"Amazon Web Services, Inc.",Louisiana+2 locations,Posted30+ days ago,2022-05-15,\nHands on experience working on large-scale d...,https://www.indeed.com/rc/clk?jk=fa378ed9974cf...,"\n\nBachelor’s degree in Computer Science, Eng..."
2,Senior Data Analyst,Tulane University,"New Orleans, LA",Posted24 days ago,2022-05-15,\nHighly proficient data analysis and data man...,https://www.indeed.com/rc/clk?jk=d35ca92e7e919...,The Data Engineer I is responsible for the ins...
3,Senior Data Analyst,IBM,"Baton Rouge, LA 70801+3 locations",Posted16 days ago,2022-05-15,\nYou analyze and organize the data needed and...,https://www.indeed.com/rc/clk?jk=745503c19eb59...,"\n\n\nIntroduction\n At IBM, work is more than..."
4,Senior Data Analyst,Pearson,"Baton Rouge, LA 70806",Posted30+ days ago,2022-05-15,"\nDesign, code, unit test, and deploy data pro...",https://www.indeed.com/rc/clk?jk=6671e57881230...,Senior Data Engineer\nThis is a Hybrid Role - ...
5,Senior Data Analyst,First Horizon Bank,"Lafayette, LA 70501",Posted30+ days ago,2022-05-15,"\nWork closely with other data engineers, busi...",https://www.indeed.com/rc/clk?jk=654bef735e60e...,\n\n\nYou will be responsible for aggregating ...
6,Senior Data Analyst,Blue Cross Blue Shield of Louisiana,"Baton Rouge, LA 70806+1 location",Posted12 days ago,2022-05-15,\nCompletion of a data engineer or relevant in...,https://www.indeed.com/rc/clk?jk=8143dc33b65a4...,We take great strides to ensure our employees ...
7,Senior Data Analyst,Deloitte,"New Orleans, LA 70139 (Central Business Distri...",Posted3 days ago,2022-05-15,"\nCollaborate with fellow engineers, engineeri...",https://www.indeed.com/rc/clk?jk=5ac34f486a9ce...,"\n\n Are you an experienced, passionate pione..."
8,Senior Data Analyst,Lucid,"Remote in New Orleans, LA",Posted30+ days ago,2022-05-15,\nSupport the business teams and product manag...,https://www.indeed.com/rc/clk?jk=4640c25ec175d...,\n\n\n\n Lucid (a Cint Group company) is a ...
9,Senior Data Analyst,Avanade,"New Orleans, LA",Posted5 days ago,2022-05-15,\nUse your knowledge to plan and deliver data ...,https://www.indeed.com/rc/clk?jk=ac7b0edc999ef...,\n About Avanade’s Data Engineering academy \n...
